In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Path ke direktori dataset
train_dir = '../datasets/train_all_set'
validation_dir = '../datasets/test_all_set'

In [ ]:
# Load model MobileNet: Muat model MobileNet yang sudah dilatih sebelumnya (pre-trained)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last few layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Tambahkan beberapa lapisan di atas MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)  # Pastikan output layer ini sesuai dengan input SVM
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Pemrosesan Data
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

In [ ]:
# Load model MobileNet: Muat model MobileNet yang sudah dilatih sebelumnya (pre-trained)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all the layers
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last few layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Tambahkan beberapa lapisan di atas MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)  # Pastikan output layer ini sesuai dengan input SVM
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Pemrosesan Data
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Found 2886 images belonging to 6 classes.
Found 180 images belonging to 6 classes.
Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 6), output.shape=(None, 256)

: 

In [ ]:
# Ekstrak fitur: Gunakan lapisan konvolusional terakhir dari MobileNet sebagai ekstraktor fitur.
feature_extractor = Model(inputs=base_model.input, outputs=x)  # Model untuk ekstraksi fitur tanpa lapisan output softmax

def extract_features(generator, model):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * generator.batch_size >= generator.samples:
            break
    return np.vstack(features), np.vstack(labels)

In [ ]:
train_features, train_labels = extract_features(train_generator, model)
validation_features, validation_labels = extract_features(validation_generator, model)

In [ ]:
# Klasifikasi dengan SVM
svm = SVC(kernel='linear', C=1.0, gamma='auto', probability=True)
svm.fit(train_features, np.argmax(train_labels, axis=1))

In [ ]:
# Evaluasi
validation_predictions = svm.predict(validation_features)
validation_true_labels = np.argmax(validation_labels, axis=1)

accuracy = accuracy_score(validation_true_labels, validation_predictions)
precision = precision_score(validation_true_labels, validation_predictions, average='weighted')
recall = recall_score(validation_true_labels, validation_predictions, average='weighted')
f1 = f1_score(validation_true_labels, validation_predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
# Prediksi dengan gambar baru
def predict_image(image_path, model, svm_model, class_indices):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    features = model.predict(img_array)
    features = features.reshape((features.shape[0], -1))

    prediction = svm_model.predict(features)
    prediction_proba = svm_model.predict_proba(features)
    predicted_class = list(class_indices.keys())[list(class_indices.values()).index(prediction[0])]

    # Tampilkan gambar dan prediksi
    plt.imshow(load_img(image_path))
    plt.title(f'Predicted class: {predicted_class}')
    plt.axis('off')
    plt.show()

    # Tampilkan nilai confidence untuk setiap kelas
    for class_name, prob in zip(class_indices.keys(), prediction_proba[0]):
        print(f'{class_name}: {prob:.4f}')

    return predicted_class, prediction_proba

# Contoh penggunaan untuk 6 gambar
image_paths = [
    '../datasets/test/test/0003_01_08_03_70.jpg',
    '../datasets/test/test/left.jpeg',
    '../datasets/test/test/OIP.jpeg',
    '../datasets/test/test/right.jpeg',
    '../datasets/test/test/1000_F_223872161_YZ98dGUBv8Oh7ce3WgF4JRbT9u07Nv4m.jpg',
    '../datasets/test/test/closed eyes.jpeg'
]

for image_path in image_paths:
    print(f'Predictions for {image_path}:')
    prediction, probabilities = predict_image(image_path, feature_extractor, svm, class_indices)
    print(f'Predicted class: {prediction}\n')